In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

# Any results you write to the current directory are saved as output.

In [ ]:
Train = pd.read_csv('../input/nyc-taxi-trip-duration/train.csv', parse_dates=True)
Test = pd.read_csv('../input/nyc-taxi-trip-duration/test.csv', parse_dates=True)
TrainSpeed = pd.read_csv('../input/estimated-speed-using-fastest-route/train_estimated_speed.csv', parse_dates=True)
TestSpeed = pd.read_csv('../input/estimated-speed-using-fastest-route/test_estimated_speed.csv', parse_dates=True)
TrainDistance = pd.read_csv('../input/new-york-city-taxi-trip-distance-matrix/train_distance_matrix.csv', parse_dates=True)
Weather = pd.read_csv('../input/weather-data-in-new-york-city-2016/weather_data_nyc_centralpark_2016(1).csv', parse_dates=True)



In [ ]:
Test.shape

In [ ]:
Weather = Weather.replace('T', '0')

In [ ]:
Train.head()

In [ ]:
Train['pickup_datetime'] = pd.to_datetime(Train['pickup_datetime'])
Test['pickup_datetime'] = pd.to_datetime(Test['pickup_datetime'])
Weather['date'] = pd.to_datetime(Weather['date'])

Train['month'] = Train['pickup_datetime'].dt.month
Train['day'] = Train['pickup_datetime'].dt.day
Train['weekday'] = Train['pickup_datetime'].dt.weekday
Train['hour'] = Train['pickup_datetime'].dt.hour
Train['minute'] = Train['pickup_datetime'].dt.minute

Test['month'] = Test['pickup_datetime'].dt.month
Test['day'] = Test['pickup_datetime'].dt.day
Test['weekday'] = Test['pickup_datetime'].dt.weekday
Test['hour'] = Test['pickup_datetime'].dt.hour
Test['minute'] = Test['pickup_datetime'].dt.minute

Weather['day'] = Weather['date'].dt.day
Weather['month'] = Weather['date'].dt.month
Weather['weekday'] = Weather['date'].dt.weekday

In [ ]:
Train = pd.merge(Train, Weather[['maximum temperature','minimum temperature', 'average temperature', 'precipitation', 'snow fall', 'snow depth', 'month', 'day', 'weekday']], on = ['month', 'day', 'weekday'], how = 'left')
Test = pd.merge(Test, Weather[['maximum temperature','minimum temperature', 'average temperature', 'precipitation', 'snow fall', 'snow depth', 'month', 'day', 'weekday']], on = ['month', 'day', 'weekday'], how = 'left')

In [ ]:
Train = pd.merge(Train, TrainSpeed[['id', 'estimated_speed']], on = ['id'], how= 'left')
Test = pd.merge(Test, TestSpeed[['id', 'estimated_speed']], on = ['id'], how= 'left')

In [ ]:
radius = 6371

Train['dlat'] = np.radians(Train['dropoff_latitude'] - Train['pickup_latitude'])
Train['dlon'] = np.radians(Train['dropoff_longitude'] - Train['pickup_longitude'])

Test['dlat'] = np.radians(Test['dropoff_latitude'] - Test['pickup_latitude'])
Test['dlon'] = np.radians(Test['dropoff_longitude'] - Test['pickup_longitude'])


Train['a'] = np.sin(Train['dlat']/2) * np.sin(Train['dlat']/2) + np.cos(np.radians(Train['pickup_latitude'])) \
        * np.cos(np.radians(Train['dropoff_latitude'])) * np.sin(Train['dlon']/2) * np.sin(Train['dlon']/2)
Train['c'] = 2 * np.arctan2(np.sqrt(Train['a']), np.sqrt(1-Train['a']))
Train['distance'] = (radius * Train['c']) * 1000

Test['a'] = np.sin(Test['dlat']/2) * np.sin(Test['dlat']/2) + np.cos(np.radians(Test['pickup_latitude'])) \
        * np.cos(np.radians(Test['dropoff_latitude'])) * np.sin(Test['dlon']/2) * np.sin(Test['dlon']/2)
Test['c'] = 2 * np.arctan2(np.sqrt(Test['a']), np.sqrt(1-Test['a']))
Test['distance'] = (radius * Test['c']) * 1000

Train = Train.drop(['dlat', 'dlon', 'a', 'c'], axis = 1)
Test = Test.drop(['dlat', 'dlon', 'a', 'c'], axis = 1)

In [ ]:
le = LabelEncoder()
le.fit(Train['store_and_fwd_flag'])
Train['store_and_fwd_flag'] = le.transform(Train['store_and_fwd_flag'])
Test['store_and_fwd_flag'] = le.transform(Test['store_and_fwd_flag'])

In [ ]:
Train.dtypes

In [ ]:
Train['snow fall'] = Train['snow fall'].astype(float)
Train['snow depth'] = Train['snow depth'].astype(float)
Train['precipitation'] = Train['precipitation'].astype(float)

Test['snow fall'] = Test['snow fall'].astype(float)
Test['snow depth'] = Test['snow depth'].astype(float)
Test['precipitation'] = Test['precipitation'].astype(float)

In [ ]:
Train['trip_duration'] = np.log(Train['trip_duration'])

In [ ]:
Train = Train.drop(['pickup_datetime', 'dropoff_datetime'], axis=1)
Test = Test.drop('pickup_datetime', axis=1)

In [ ]:
Train.columns

In [ ]:
Test.dtypes

In [ ]:
Test_orignal = Test.drop('id', axis=1)

In [ ]:
len(Train.columns), len(Test.columns)

Train['estimated_speed'].isnull().sum()
Train['estimated_speed'] = Train['estimated_speed'].fillna(Train['estimated_speed'].median())

Train[[ 'vendor_id', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'month', 'day', 'weekday',
       'hour', 'minute', 'maximum temperature', 'minimum temperature',
       'average temperature', 'precipitation', 'snow fall', 'snow depth',
       'estimated_speed', 'distance']] = Train[[ 'vendor_id', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'month', 'day', 'weekday',
       'hour', 'minute', 'maximum temperature', 'minimum temperature',
       'average temperature', 'precipitation', 'snow fall', 'snow depth',
       'estimated_speed', 'distance']].astype(float)

In [ ]:
X = Train.drop(['id', 'trip_duration'], axis = 1)
y = Train['trip_duration'].values

In [ ]:
#xtrain = Train.drop(['id', 'trip_duration'], axis = 1).as_matrix
#xtest = Test.drop(['id' ], axis = 1).as_matrix()
#ytrain = Train['trip_duration'].values
#id_train = Train['id'].values
#id_test = Test['id'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=42)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error


mean_train = np.mean(y_train)

baseline_predictions = np.ones(y_test.shape) * mean_train

mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))


params = {
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
     'objective':'reg:linear',}

params['eval_metric'] = "mae"


num_boost_round = 999


model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10,
)

cv_results['test-mae-mean'].min()


gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

min_mae = float("Inf")
best_params = None

for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
   
params['max_depth'] = max_depth
params['min_child_weight'] = min_child_weight
    
cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10)
        
mean_mae = cv_results['test-mae-mean'].min()
 boost_rounds = cv_results['test-mae-mean'].argmin()
   
print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
   
if mean_mae < min_mae:
        min_mae = mean_mae

best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))


gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

min_mae = float("Inf")
best_params = None

for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))

params['subsample'] = subsample
params['colsample_bytree'] = colsample

cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10)
        

mean_mae = cv_results['test-mae-mean'].min()
boost_rounds = cv_results['test-mae-mean'].argmin()
print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))

if mean_mae < min_mae:
        min_mae = mean_mae

best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

params['subsample'] = 1
params['colsample_bytree'] = 1.

min_mae = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))

params['eta'] = eta
 
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics=['mae'],
    early_stopping_rounds=10)
   
mean_mae = cv_results['test-mae-mean'].min()
boost_rounds = cv_results['test-mae-mean'].argmin()

print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))

if mean_mae < min_mae:
    min_mae = mean_mae

best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

In [ ]:
params = {'max_depth': 11,
 'min_child_weight': 7,
 'eta': 0.05,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'objective': 'reg:linear',
 'eval_metric': 'mae'}

num_boost_round = 3500

In [ ]:
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test"),],
    early_stopping_rounds=20
)

In [ ]:
mean_absolute_error(best_model.predict(dtest), y_test)


In [ ]:
BestPrediction = np.exp(best_model.predict(xgb.DMatrix(Test_orignal)))

In [ ]:
output = pd.DataFrame()
output['id'] = Test['id']
output['trip_duration'] = BestPrediction
output.to_csv('FinalSubmission.csv', index=False)